In [ ]:
import pandas as pd
import numpy as np

import platform
import matplotlib.pyplot as plt

# Warning 메세지 무시
import warnings
warnings.filterwarnings(module='sklearn*', action='ignore', category=DeprecationWarning)

# 1. 재학생 중도탈락 예측모형 개발
## 1-1) 과거(前) 재학생데이터 수집
* 엑셀(excel)에 저장되어 있는 예측모형구축용 과거 재학생 원천데이터셋을 데이터프레임으로 읽어들임
* 과거 재학생은 21/22학년도 2개년 재학생(2년제 1학년, 3년제 1,2학년)이며, 데이터수집 기준시점은 당해 9월 1일이고 중도탈락여부는 다음해 4월 1일 기준임

In [ ]:
# 예측모형구축용 과거 재학생 원천데이터셋을 데이터프레임으로 읽어들임
rawData_org = pd.read_excel('과거_재학생_원천데이터셋.xlsx', sheet_name = '예측모형구축용')

# 과거 재학생 원천데이터셋을 저장하는 데이터프레임 확인
rawData_org

,학생ID,소속학과,학년,성별,나이,거주지,학점,F학점과목수,결석수,장학건수,마일리지점수,지도교수상담횟수,전문가상담횟수,대학생활적응분류,중도탈락여부
0,8882,학과01,2,남,23,거주001,3.06,0,0.000,0,27,3,0,중점지도,0
1,7377,학과02,1,남,24,거주002,2.05,1,0.750,0,20,2,0,NaN,0
2,8851,학과03,1,여,23,거주003,4.45,0,0.103,3,1,0,1,중점지도,0
3,8852,학과04,2,남,23,거주004,3.40,0,0.400,0,6,2,0,중점지도,1
4,109,학과03,2,남,23,거주005,3.38,0,1.083,0,25,2,1,중점지도,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6533,2019,학과08,2,여,20,거주066,2.92,0,1.105,0,19,4,1,관심지도,0
6534,11050,학과01,2,남,22,거주013,2.55,0,0.421,0,12,3,0,중점지도,0
6535,5179,학과01,2,남,22,거주062,2.50,0,1.053,2,12,3,0,일반지도,0
6536,801,학과01,2,남,22,거주021,4.50,0,0.000,3,7,1,1,중점지도,0


## 1-2) 예측모형구축용 데이터셋 EDA 및 전처리
(1) 원천데이터셋을 구성하는 각 변수(특성)의 값 분포를 파악<br>
(2) 원천데이터셋에 존재하는 결측치(missing value) 처리<br>
(3) 원천데이터셋을 구성하는 각 변수 간의 상관 관계를 파악<br>
(4) 원천데이터셋의 변수가 너무 많을 경우, 분석차원의 축소(또는 제거)<br>
(5) 범주형 특성(categrical feature)에 대한 인코딩<br>

### (1-2-1) 원천데이터셋을 구성하는 각 변수(특성)의 값 분포 파악

In [ ]:
# NULL값의 개수 구하기
rawData_org.isnull().sum()

학생ID          0
소속학과          0
학년            0
성별            0
나이            0
거주지          43
학점          106
F학점과목수        0
결석수           0
장학건수          0
마일리지점수        0
지도교수상담횟수      0
전문가상담횟수       0
대학생활적응분류    336
중도탈락여부        0
dtype: int64

In [ ]:
# 수치형 컬럼 각각에 대한 건수, 평균, 표준편차, 최소값, 25분위값, 50분위값, 75분위값, 최대값 구하기
rawData_org[['나이','학점','F학점과목수','결석수','장학건수','마일리지점수','지도교수상담횟수','전문가상담횟수']].describe()

,나이,학점,F학점과목수,결석수,장학건수,마일리지점수,지도교수상담횟수,전문가상담횟수
count,6538.000000,6432.000000,6538.000000,6538.000000,6538.000000,6538.000000,6538.000000,6538.000000
mean,20.115173,3.301729,0.339706,0.588718,2.123585,20.854543,1.953961,0.468186
std,2.720476,0.850320,1.161735,1.387512,1.460622,18.518490,1.072637,0.773775
min,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,2.920000,0.000000,0.000000,1.000000,8.000000,1.000000,0.000000
50%,19.000000,3.470000,0.000000,0.143000,2.000000,15.000000,2.000000,0.000000
75%,20.000000,3.900000,0.000000,0.524000,3.000000,28.000000,2.000000,1.000000
max,77.000000,4.500000,9.000000,14.263000,18.000000,133.000000,9.000000,16.000000


In [ ]:
# 범주형 컬럼 각각에 대한 값의 분포
# 소속학과별 건수 (건수 내림차순으로 정렬)
rawData_org[['소속학과','학생ID']].groupby('소속학과').학생ID.agg(['count']).sort_values(by=['count'], ascending=False)

,count
소속학과,
학과13,452
학과14,315
학과15,311
학과18,278
학과06,270
학과07,266
학과05,262
학과08,255
학과25,253


In [ ]:
# 범주형 컬럼 각각에 대한 값의 분포
# 학년별 건수 (건수 내림차순으로 정렬)
rawData_org[['학년','학생ID']].groupby('학년').학생ID.agg(['count']).sort_values(by=['count'], ascending=False)

,count
학년,
1,4504
2,2034


In [ ]:
# 범주형 컬럼 각각에 대한 값의 분포
# 성별 건수 (건수 내림차순으로 정렬)
rawData_org[['성별','학생ID']].groupby('성별').학생ID.agg(['count']).sort_values(by=['count'], ascending=False)

,count
성별,
여,3737
남,2801


In [ ]:
# 범주형 컬럼 각각에 대한 값의 분포
# 거주지별 건수 (건수 내림차순으로 정렬)
rawData_org[['거주지','학생ID']].groupby('거주지').학생ID.agg(['count']).sort_values(by=['count'], ascending=False)

,count
거주지,
거주006,841
거주008,398
거주011,391
거주021,353
거주004,326
...,...
거주084,1
거주159,1
거주137,1


In [ ]:
# 범주형 컬럼 각각에 대한 값의 분포
# 대학생활적응분류별 건수 (건수 내림차순으로 정렬)
rawData_org[['대학생활적응분류','학생ID']].groupby('대학생활적응분류').학생ID.agg(['count']).sort_values(by=['count'], ascending=False)

,count
대학생활적응분류,
일반지도,2562
관심지도,1992
중점지도,1648


In [ ]:
# 중도탈락여부별 건수 (건수 내림차순으로 정렬)
rawData_org[['중도탈락여부','학생ID']].groupby('중도탈락여부').학생ID.agg(['count']).sort_values(by=['count'], ascending=False)

,count
중도탈락여부,
0,5398
1,1140


### (1-2-2) 원천데이터셋에 존재하는 결측치(missing value) 처리
* 총 6,538건 중 441건이 결측치로 인한 제거 대상 건(row)으로 제거 대상 건이 크지 않고 결측치를 채움으로 인한 왜곡현상도 있을 수 있어 제거 방식을 택함

In [ ]:
# 결측치 제거 (row-wise deletion)
rawData_org = rawData_org.dropna()
rawData_org

,학생ID,소속학과,학년,성별,나이,거주지,학점,F학점과목수,결석수,장학건수,마일리지점수,지도교수상담횟수,전문가상담횟수,대학생활적응분류,중도탈락여부
0,8882,학과01,2,남,23,거주001,3.06,0,0.000,0,27,3,0,중점지도,0
2,8851,학과03,1,여,23,거주003,4.45,0,0.103,3,1,0,1,중점지도,0
3,8852,학과04,2,남,23,거주004,3.40,0,0.400,0,6,2,0,중점지도,1
4,109,학과03,2,남,23,거주005,3.38,0,1.083,0,25,2,1,중점지도,0
5,8857,학과05,2,남,23,거주005,2.50,0,0.143,0,13,2,0,관심지도,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6533,2019,학과08,2,여,20,거주066,2.92,0,1.105,0,19,4,1,관심지도,0
6534,11050,학과01,2,남,22,거주013,2.55,0,0.421,0,12,3,0,중점지도,0
6535,5179,학과01,2,남,22,거주062,2.50,0,1.053,2,12,3,0,일반지도,0
6536,801,학과01,2,남,22,거주021,4.50,0,0.000,3,7,1,1,중점지도,0


In [ ]:
# 인덱스 재설정
rawData_org.reset_index(inplace=True, drop=True)
rawData_org

,학생ID,소속학과,학년,성별,나이,거주지,학점,F학점과목수,결석수,장학건수,마일리지점수,지도교수상담횟수,전문가상담횟수,대학생활적응분류,중도탈락여부
0,8882,학과01,2,남,23,거주001,3.06,0,0.000,0,27,3,0,중점지도,0
1,8851,학과03,1,여,23,거주003,4.45,0,0.103,3,1,0,1,중점지도,0
2,8852,학과04,2,남,23,거주004,3.40,0,0.400,0,6,2,0,중점지도,1
3,109,학과03,2,남,23,거주005,3.38,0,1.083,0,25,2,1,중점지도,0
4,8857,학과05,2,남,23,거주005,2.50,0,0.143,0,13,2,0,관심지도,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6092,2019,학과08,2,여,20,거주066,2.92,0,1.105,0,19,4,1,관심지도,0
6093,11050,학과01,2,남,22,거주013,2.55,0,0.421,0,12,3,0,중점지도,0
6094,5179,학과01,2,남,22,거주062,2.50,0,1.053,2,12,3,0,일반지도,0
6095,801,학과01,2,남,22,거주021,4.50,0,0.000,3,7,1,1,중점지도,0


### (1-2-3) 원천데이터셋을 구성하는 각 변수 간의 상관 관계를 파악
* 독립변수와 종속변수(중도탈락여부)와의 상관관계 파악
 * 수치형 독립변수 : 로지스틱 회귀분석
  * 결정계수가 1에 가까울수록 계수의 절대값이 클수록 p값이 낮을수록 상관관계가 높음 (계수값이 양수이면 양의 상관관계, 음수이면 음의 상관계)
 * 범주형 독립변수 : 카이제곱 검정
  * 측정한 카이제곱통계량과 자유도를 카이제곱분포표에 적용한 결과 도출된 유의확률이 미리 설정한 유의수준(상관성의 판단기준)보다 작을 때 상관성이 있다고 판단
* 독립변수들 간의 다중공선성 파악 : 분산팽창계수
 * 다중공선성 : 하나의 독립변수가 다른 여러 개의 독립변수들로 잘 예측되는 성질
  * 분산팽창계수(VIF, Variance Inflation Factor)다)가 보통 10보다 크면 다중공선성이 있다고 판단 -> 다중공선성이 있는 독립변수는 예측모형의 최종 독립변수에서 제외

In [ ]:
# 수치형 독립변수와 종속변수(중도탈락여부)와의 상관성 파악을 위한 로지스틱 회귀분석(Logistic Regression)
# features : 독립변수 데이터프레임
# dropout : 종속변수 리스트
features = rawData_org[['나이','학점','F학점과목수','결석수','장학건수','마일리지점수','지도교수상담횟수','전문가상담횟수']]
dropout = rawData_org['중도탈락여부']

In [ ]:
# train/test set 분리하기
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, dropout)

In [ ]:
# 데이터 정규화(표준화) : 각 독립변수의 scale이 달라서 생길 수 있는 종속변수에 끼칠 영향도의 차이를 없애기 위해 동일 scale로 표준화 진행
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.fit_transform(X_test)

In [ ]:
# 로지스틱 분석 시행
import statsmodels.api as sm

logreg = sm.Logit(y_train, sm.add_constant(X_train_std)).fit()
logreg.summary()

Optimization terminated successfully.
         Current function value: 0.392532
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 중도탈락여부   No. Observations:                 4572
Model:                          Logit   Df Residuals:                     4563
Method:                           MLE   Df Model:                            8
Date:                Sat, 06 Jul 2024   Pseudo R-squ.:                  0.1184
Time:                        18:07:01   Log-Likelihood:                -1794.7
converged:                       True   LL-Null:                       -2035.6
Covariance Type:            nonrobust   LLR p-value:                 5.163e-99
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.9125      0.051    -37.780      0.000      -2.012      -1.813
x1            -0.9548      0.110     -8.701      0.000      -1.170      -0.740
x2            -0.3852      0.064     -6.024      0.000      -0.511      -0.260
x3            -0.0485      0.074     -0.652      0.515      -0.194       0.097
x4             0.3504      0.081      4.345      0.000       0.192       0.508
x5             0.0901      0.044      2.035      0.042       0.003       0.177
x6            -0.2992      0.056     -5.364      0.000      -0.409      -0.190
x7             0.1692      0.047      3.568      0.000       0.076       0.262
x8            -0.2334      0.059     -3.928      0.000      -0.350      -0.117
==============================================================================
"""

* '나이'(x1),'학점'(x2),'F학점과목수'(x3),'결석수'(x4),'장학건수'(x5),'마일리지점수'(x6),'지도교수상담횟수'(x7),'전문가상담횟수'(x8)
* (유사)결정계수(Pseudo R-squ.) 값으로 미루어 모델의 설명력은 약한 편임
* coef는 각 독립변수의 계수로 '나이'(x1),'학점'(x2),'F학점과목수'(x3),'마일리지점수'(x6),'전문가상담횟수'(x8)의 값이 클수록 '결석수'(x4),'장학건수'(x5),'지도교수상담횟수'(x7)의 값이 작을수록 중도탈락을 하지 않은 경향성을 띰
* P-value 관점에서는 'F학점과목수'(x3),'장학건수'(x5)의 상대적으로 설명력이 크게 약함
* 로지스틱 분석 통계 상으로는 신뢰도 높은 강건한 모델로 보기는 어려움

In [ ]:
# 로지스틱 회귀모델 생성 및 평가
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_std, y_train)

LogisticRegression()

In [ ]:
print(model.score(X_train_std, y_train))
print(model.score(X_test_std, y_test))

0.8466754155730534
0.8327868852459016


* 트레이닝셋과 테스트셋에서 정확도는 높은 편이나 정확도만으로 모델 성능을 속단하기는 이름

In [ ]:
# 범주형 독립변수와 종속변수(중도탈락여부)와의 상관성 파악을 위한 카이제곱 검정(Chi-square Validation)
# 카이제곱검정을 위한 패키지 임포트
from scipy.stats import chi2_contingency

# 소속학과와 중도탈락여부 상관성 파악
# 검증함수 활용을 위한 데이터셋 재구조화
features = rawData_org[['소속학과','중도탈락여부','학생ID']]
cnt_df_1 = features.loc[features['중도탈락여부']==1].groupby(['소속학과','중도탈락여부']).학생ID.agg(['count'])
cnt_df_0 = features.loc[features['중도탈락여부']==0].groupby(['소속학과','중도탈락여부']).학생ID.agg(['count'])
cnt_df = pd.merge(cnt_df_1,cnt_df_0, how='inner',on='소속학과')
cnt_df.rename(columns={'count_x':'탈락','count_y':'지속'}, inplace=True)

# 검증함수 수행
chiresult = chi2_contingency(cnt_df, correction=False)
print('(소속학과, 중도탈락여부)')
print('Chi square: {}'.format(chiresult[0]))
print('p-value: {}'.format(chiresult[1]))
if chiresult[1] <= 0.05 :  # 유의수준 설정은 사용자정의인데, 대체로 0.05로 설정
    print(' -> 유의확률(p-value)이 유의수준(0.05) 이하이므로 소속학과는 중도탈락여부와 상관성이 있다')
else :
    print(' -> 유의확률(p-value)이 유의수준(0.05) 초과이므로 소속학과는 중도탈락여부와 상관성이 없다(독립변수에서 제외 검토)')

(소속학과, 중도탈락여부)
Chi square: 298.8671118318969
p-value: 3.70183691256242e-43
 -> 유의확률(p-value)이 유의수준(0.05) 이하이므로 소속학과는 중도탈락여부와 상관성이 있다


In [ ]:
# 범주형 독립변수와 종속변수(중도탈락여부)와의 상관성 파악을 위한 카이제곱 검정(Chi-square Validation)
# 카이제곱검정을 위한 패키지 임포트
from scipy.stats import chi2_contingency

# 소속학과와 중도탈락여부 상관성 파악
# 검증함수 활용을 위한 데이터셋 재구조화
features = rawData_org[['학년','중도탈락여부','학생ID']]
cnt_df_1 = features.loc[features['중도탈락여부']==1].groupby(['학년','중도탈락여부']).학생ID.agg(['count'])
cnt_df_0 = features.loc[features['중도탈락여부']==0].groupby(['학년','중도탈락여부']).학생ID.agg(['count'])
cnt_df = pd.merge(cnt_df_1,cnt_df_0, how='inner',on='학년')
cnt_df.rename(columns={'count_x':'탈락','count_y':'지속'}, inplace=True)

# 검증함수 수행
chiresult = chi2_contingency(cnt_df, correction=False)
print('(학년, 중도탈락여부)')
print('Chi square: {}'.format(chiresult[0]))
print('p-value: {}'.format(chiresult[1]))
if chiresult[1] <= 0.05 :  # 유의수준 설정은 사용자정의인데, 대체로 0.05로 설정
    print(' -> 유의확률(p-value)이 유의수준(0.05) 이하이므로 학년은 중도탈락여부와 상관성이 있다')
else :
    print(' -> 유의확률(p-value)이 유의수준(0.05) 초과이므로 학년은 중도탈락여부와 상관성이 없다(독립변수에서 제외 검토)')

(학년, 중도탈락여부)
Chi square: 188.0696927875707
p-value: 8.388541507992485e-43
 -> 유의확률(p-value)이 유의수준(0.05) 이하이므로 학년은 중도탈락여부와 상관성이 있다


In [ ]:
# 범주형 독립변수와 종속변수(중도탈락여부)와의 상관성 파악을 위한 카이제곱 검정(Chi-square Validation)
# 카이제곱검정을 위한 패키지 임포트
from scipy.stats import chi2_contingency

# 소속학과와 중도탈락여부 상관성 파악
# 검증함수 활용을 위한 데이터셋 재구조화
features = rawData_org[['성별','중도탈락여부','학생ID']]
cnt_df_1 = features.loc[features['중도탈락여부']==1].groupby(['성별','중도탈락여부']).학생ID.agg(['count'])
cnt_df_0 = features.loc[features['중도탈락여부']==0].groupby(['성별','중도탈락여부']).학생ID.agg(['count'])
cnt_df = pd.merge(cnt_df_1,cnt_df_0, how='inner',on='성별')
cnt_df.rename(columns={'count_x':'탈락','count_y':'지속'}, inplace=True)

# 검증함수 수행
chiresult = chi2_contingency(cnt_df, correction=False)
print('(학년, 중도탈락여부)')
print('Chi square: {}'.format(chiresult[0]))
print('p-value: {}'.format(chiresult[1]))
if chiresult[1] <= 0.05 :  # 유의수준 설정은 사용자정의인데, 대체로 0.05로 설정
    print(' -> 유의확률(p-value)이 유의수준(0.05) 이하이므로 성별은 중도탈락여부와 상관성이 있다')
else :
    print(' -> 유의확률(p-value)이 유의수준(0.05) 초과이므로 성별은 중도탈락여부와 상관성이 없다(독립변수에서 제외 검토)')

(학년, 중도탈락여부)
Chi square: 447.22355858833413
p-value: 2.8996806343115407e-99
 -> 유의확률(p-value)이 유의수준(0.05) 이하이므로 성별은 중도탈락여부와 상관성이 있다


In [ ]:
# 범주형 독립변수와 종속변수(중도탈락여부)와의 상관성 파악을 위한 카이제곱 검정(Chi-square Validation)
# 카이제곱검정을 위한 패키지 임포트
from scipy.stats import chi2_contingency

# 소속학과와 중도탈락여부 상관성 파악
# 검증함수 활용을 위한 데이터셋 재구조화
features = rawData_org[['거주지','중도탈락여부','학생ID']]
cnt_df_1 = features.loc[features['중도탈락여부']==1].groupby(['거주지','중도탈락여부']).학생ID.agg(['count'])
cnt_df_0 = features.loc[features['중도탈락여부']==0].groupby(['거주지','중도탈락여부']).학생ID.agg(['count'])
cnt_df = pd.merge(cnt_df_1,cnt_df_0, how='inner',on='거주지')
cnt_df.rename(columns={'count_x':'탈락','count_y':'지속'}, inplace=True)

# 검증함수 수행
chiresult = chi2_contingency(cnt_df, correction=False)
print('(학년, 중도탈락여부)')
print('Chi square: {}'.format(chiresult[0]))
print('p-value: {}'.format(chiresult[1]))
if chiresult[1] <= 0.05 :  # 유의수준 설정은 사용자정의인데, 대체로 0.05로 설정
    print(' -> 유의확률(p-value)이 유의수준(0.05) 이하이므로 거주지는 중도탈락여부와 상관성이 있다')
else :
    print(' -> 유의확률(p-value)이 유의수준(0.05) 초과이므로 거주지는 중도탈락여부와 상관성이 없다(독립변수에서 제외 검토)')

(학년, 중도탈락여부)
Chi square: 92.71638040761017
p-value: 0.5180839954208329
 -> 유의확률(p-value)이 유의수준(0.05) 초과이므로 거주지는 중도탈락여부와 상관성이 없다(독립변수에서 제외 검토)


In [ ]:
# 범주형 독립변수와 종속변수(중도탈락여부)와의 상관성 파악을 위한 카이제곱 검정(Chi-square Validation)
# 카이제곱검정을 위한 패키지 임포트
from scipy.stats import chi2_contingency

# 소속학과와 중도탈락여부 상관성 파악
# 검증함수 활용을 위한 데이터셋 재구조화
features = rawData_org[['대학생활적응분류','중도탈락여부','학생ID']]
cnt_df_1 = features.loc[features['중도탈락여부']==1].groupby(['대학생활적응분류','중도탈락여부']).학생ID.agg(['count'])
cnt_df_0 = features.loc[features['중도탈락여부']==0].groupby(['대학생활적응분류','중도탈락여부']).학생ID.agg(['count'])
cnt_df = pd.merge(cnt_df_1,cnt_df_0, how='inner',on='대학생활적응분류')
cnt_df.rename(columns={'count_x':'탈락','count_y':'지속'}, inplace=True)

# 검증함수 수행
chiresult = chi2_contingency(cnt_df, correction=False)
print('(학년, 중도탈락여부)')
print('Chi square: {}'.format(chiresult[0]))
print('p-value: {}'.format(chiresult[1]))
if chiresult[1] <= 0.05 :  # 유의수준 설정은 사용자정의인데, 대체로 0.05로 설정
    print(' -> 유의확률(p-value)이 유의수준(0.05) 이하이므로 대학생활적응분류는 중도탈락여부와 상관성이 있다')
else :
    print(' -> 유의확률(p-value)이 유의수준(0.05) 초과이므로 대학생활적응분류는 중도탈락여부와 상관성이 없다(독립변수에서 제외 검토)')

(학년, 중도탈락여부)
Chi square: 45.97483506240162
p-value: 1.0391815152545218e-10
 -> 유의확률(p-value)이 유의수준(0.05) 이하이므로 대학생활적응분류는 중도탈락여부와 상관성이 있다


In [ ]:
# 분산팽창계수를 구하기 위한 패키지(모듈) 임포트
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 다중공선성 파악을 위한 분산팽창계수 구하기
data = rawData_org[['나이','학점','F학점과목수','결석수','장학건수','마일리지점수','지도교수상담횟수','전문가상담횟수']]

# 데이터 정규화(표준화) : 각 독립변수의 scale이 달라서 생길 수 있는 종속변수에 끼칠 영향도의 차이를 없애기 위해 동일 scale로 표준화 진행
from sklearn.preprocessing import StandardScaler

# 정규화(표준화) 수행
scaler = StandardScaler()
data_std = scaler.fit_transform(data)

# VIF 출력을 위한 데이터 프레임 형성
vif = pd.DataFrame()

# VIF 값과 각 Feature 이름에 대해 설정
vif["VIF Factor"] = [variance_inflation_factor(data_std, i) for i in range(data_std.shape[1])]
vif["features"] = data.columns

# VIF 값이 높은 순으로 정렬
vif = vif.sort_values(by="VIF Factor", ascending=False)
vif = vif.reset_index().drop(columns='index')
vif

,VIF Factor,features
0,4.369904,결석수
1,4.223410,F학점과목수
2,2.246277,학점
3,1.185514,지도교수상담횟수
4,1.171872,마일리지점수
5,1.064749,전문가상담횟수
6,1.027617,장학건수
7,1.009026,나이


* 각 독립변수에 대해서 분산팽창계수가 10 미만이므로 모두 유의미한 독립변수로 간주할 수 있음

### (1-2-4) 분석차원의 축소(또는 제거)
* 특성 선택(feature selection) : 여러 특성(변수) 중에서 종속변수에 영향도가 높은 일부 특성군을 선택하여 최종 독립변수로 활용
 * 로지스틱 회귀분석 결과 p값이 작고 계수(coef)의 절대값이 큰 변수 위주로 선택 가능
  * 본 예제에서 '나이'(x1),'학점'(x2),'결석수'(x4),'마일리지점수'(x6),'전문가상담횟수'(x8) 등이 우선 고려 대상
* 특성 추출(feature extraction) : 여러 특성(변수)를 합성하여 저차원의 새로운 특성을 추출해서 최종 독립변수로 활용
  * 대표적으로 주성분분석(PCA) 기법이 있음

In [ ]:
# 주성분분석
# 주성분분석을 위한 수치형 독립변수 선택 (나이를 제외하고 교과, 비교과, 장학, 상담과 관련 있는 변수 선택)
data = rawData_org[['학점','F학점과목수','결석수','장학건수','마일리지점수','지도교수상담횟수','전문가상담횟수']]

# 데이터 정규화(표준화) : 각 독립변수의 scale이 달라서 생길 수 있는 종속변수에 끼칠 영향도의 차이를 없애기 위해 동일 scale로 표준화 진행
from sklearn.preprocessing import StandardScaler

# 정규화(표준화) 수행
scaler = StandardScaler()
data_std = scaler.fit_transform(data)

features = ['학점','F학점과목수','결석수','장학건수','마일리지점수','지도교수상담횟수','전문가상담횟수']
pd.DataFrame(data_std, columns=features)

# 주성분분석을 위한 패키지(모듈) 임포트
from sklearn.decomposition import PCA

# 주성분 모형변수
pca = PCA(n_components=7)

# 원속성을 주성분으로 변환
printcipalComponents = pca.fit_transform(data_std)
principalDf = pd.DataFrame(data=printcipalComponents, columns = ['1', '2', '3','4','5','6','7'])

# 각 주성분의 설명력 출력 (설명력의 누적값을 활용하여 주성분의 갯수 결정. 대체로 90~95% 이상에서 결정)
print(pca.explained_variance_ratio_)

[0.37201082 0.18793299 0.14886958 0.14054359 0.08422712 0.04716576
 0.01925014]


* 설명력(설명가능한 분산량)이 0.9 이상이 되려면 5개의 주성분(PC) 필요
 * 0.37201082 + 0.18793299 + 0.14886958 + 0.14054359 + 0.08422712 = 0.9335841

In [ ]:
# 주성분을 5개로 결정 (설명력 : 93.4%)
pca = PCA(n_components=5)

printcipalComponents = pca.fit_transform(data_std)
principalDf = pd.DataFrame(data=printcipalComponents, columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])
principalDf

,PC1,PC2,PC3,PC4,PC5
0,-0.246850,1.157398,-0.765821,-1.365021,0.216904
1,-0.774082,-2.399318,-0.022895,0.462774,-0.307951
2,0.028404,-0.112391,-1.372695,-1.197276,0.224339
3,0.197683,-0.025076,0.222300,-1.630282,0.132367
4,0.435994,0.157623,-1.176404,-1.286726,0.062835
...,...,...,...,...,...
6092,0.372368,1.224690,0.073522,-1.575545,1.607968
6093,0.447191,0.831843,-1.173926,-1.249884,0.710430
6094,0.647441,0.803695,-0.743739,0.103049,0.741016
6095,-1.017927,-1.563868,0.158279,0.428400,0.130106


* 7개의 원속성이 5개의 주성분으로 변환되어 분석차원 축소효과를 거둠 (설명력 : 약 93.4%)

In [ ]:
# 각 주성분(PC) 값을 결정하는데 있어서 원속성들의 영향력 파악 (주성분의 특성 파악을 위해 필요)
pca_df = pd.DataFrame(pca.components_, columns=['학점','F학점과목수','결석수','장학건수','마일리지점수','지도교수상담횟수','전문가상담횟수'])
pca_df.index = ['PC1','PC2','PC3','PC4','PC5']
pca_df

,학점,F학점과목수,결석수,장학건수,마일리지점수,지도교수상담횟수,전문가상담횟수
PC1,-0.539322,0.568546,0.574398,-0.102603,-0.183473,-0.108305,0.005947
PC2,-0.087519,0.111147,0.105082,-0.064895,0.522980,0.730293,-0.397364
PC3,-0.007378,0.126668,0.088578,0.274926,0.555391,0.012395,0.769323
PC4,0.022812,0.086611,0.068368,0.948174,-0.131222,0.013080,-0.266232
PC5,-0.047164,-0.063585,-0.043585,0.036801,-0.598846,0.672774,0.423364


* PC1은 학점, F학점과목수, 결석수를 상대적으로 많이 반영
* PC2, PC3, PC5는 마일리지점수, 지도교수상담횟수, 전문가상담횟수를 상대적으로 많이 반영하는데, 반영 방향이 많이 다름
* PC4는 장학건수를 월등하게 많이 반영
* -> 주성분 별로 서로 다른 특성들이 반영되어 특징을 구분할 수 있으나, 원속성의 특성이 조금씩은 반영되어 있어서 주성분으로 예측모형을 구축할 경우 예측 신뢰성과는 별도로 독립변수의 해석이 다소 모호할 수 있음

### (1-2-5) 범주형 특성(categrical feature)에 대한 인코딩
* 범주형 특성에 대해서 예측모형구축용과 예측용 데이터셋에서 동일한 범주값이 동일하게 인코딩됨을 보장하기 위해서 예측용 데이터셋도 미리 읽어들여 NULL값을 가진 건은 삭제하고 난 다음 예측모형구축용과 예측용 데이터셋을 한꺼번에 인코딩함

In [ ]:
# 예측용 현 입학초신입생 원천데이터셋을 데이터프레임으로 읽어들임
# rawData_org : 예측모형구축용 원천데이터셋, rawData2_org : 예측용 원천데이터셋
rawData2_org = pd.read_excel('현재_재학생_원천데이터셋.xlsx', sheet_name = '예측용')

rawData_org['구분'] = '예측모형구축용'
rawData2_org['구분'] = '예측용'

# 예측용 데이터셋에 대한 결측치 제거 (row-wise deletion) -> 결과를 rawData2d_org 데이터프레임에 저장
rawData2d_org = rawData2_org.dropna()
rawData2d_org.reset_index(inplace=True, drop=True)  # 인덱스 재설정

rawData_all = pd.concat([rawData_org, rawData2d_org])

# 범주형 특성에 대한 레이블 인코딩(label encoding)
categorical_columns = ['소속학과', '학년', '성별', '거주지', '대학생활적응분류', '중도탈락여부']

<ipython-input-255-4f329cbe5829>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rawData_org['구분'] = '예측모형구축용'


In [ ]:
# 레이블인코딩을 위한 패키지(모듈) 임포트
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for feature in categorical_columns:
    rawData_all[feature] = le.fit_transform(rawData_all[feature])

# rawData_e : 전처리된 예측모형구축용 데이터셋, rawData2_e : 전처리된 예측용 데이터셋
rawData_e = rawData_all[rawData_all['구분'] == '예측모형구축용'].iloc[:, :-1]
rawData2_e = rawData_all[rawData_all['구분'] == '예측용'].iloc[:, :-1]

## 1-3) 예측모형 설계·구축
(1) 독립변수와 종속변수의 확정<br>
(2) 분석대상데이터셋을 트레이닝셋과 테스트셋으로 분리<br>
(3) 종속변수인 중도탈락여부 클래스의 분포 불균형 해소<br>
(4) 트레이닝셋을 대상으로 분류예측알고리듬 별로 학습을 수행하고 성능 측정/비교하여 가장 우수한 알고리듬 선택<br>
(5) 독립변수 별 중도탈락예측 영향도 파악<br>

### (1-3-1) 독립변수와 종속변수의 확정
* 독립변수의 개수(차원수)가 지나치게 많다면 예측모형의 성능이 떨어지고 과적합(Overfitting)의 문제가 생길 수 있음
* -> 로지스틱회귀, 카이제곱검정, 공선성분석 등을 통해 독립적이면서 종속변수에 영향도가 높은 변수들을 선택하거나 주성분분석과 같이 차원축소 방식을 통해 독립변수의 개수를 줄일 필요 있음

In [ ]:
rawData_org

,학생ID,소속학과,학년,성별,나이,거주지,학점,F학점과목수,결석수,장학건수,마일리지점수,지도교수상담횟수,전문가상담횟수,대학생활적응분류,중도탈락여부,구분
0,8882,학과01,2,남,23,거주001,3.06,0,0.000,0,27,3,0,중점지도,0,예측모형구축용
1,8851,학과03,1,여,23,거주003,4.45,0,0.103,3,1,0,1,중점지도,0,예측모형구축용
2,8852,학과04,2,남,23,거주004,3.40,0,0.400,0,6,2,0,중점지도,1,예측모형구축용
3,109,학과03,2,남,23,거주005,3.38,0,1.083,0,25,2,1,중점지도,0,예측모형구축용
4,8857,학과05,2,남,23,거주005,2.50,0,0.143,0,13,2,0,관심지도,0,예측모형구축용
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6092,2019,학과08,2,여,20,거주066,2.92,0,1.105,0,19,4,1,관심지도,0,예측모형구축용
6093,11050,학과01,2,남,22,거주013,2.55,0,0.421,0,12,3,0,중점지도,0,예측모형구축용
6094,5179,학과01,2,남,22,거주062,2.50,0,1.053,2,12,3,0,일반지도,0,예측모형구축용
6095,801,학과01,2,남,22,거주021,4.50,0,0.000,3,7,1,1,중점지도,0,예측모형구축용


In [ ]:
# 독립변수+종속변수 확정
# 독립변수 : 소속학과, 학년, 성별, 나이, 거주지, 학점, F학점과목수, 결석수, 장학건수, 마일리지점수, 지도교수상담횟수, 전문가상담횟수,
#            대학생활적응분류
# 종속변수 : 중도탈락여부

# feature_set_shrink : EDA를 통해 유의미성이 낮거나 종속변수와의 상관성이 낮은 독립변수를 제거할 것인지 여부 (1:제거, 0:제거하지않음)
# 유의미성이 낮은 변수 : F학점과목수, 장학건수
# 상관성이 낮은 변수 : 거주지
feature_set_shrink = 0

if feature_set_shrink == 1:
    feature_list = ['소속학과', '학년', '성별', '나이', '학점', '결석수',
                              '마일리지점수', '지도교수상담횟수', '전문가상담횟수', '대학생활적응분류', '중도탈락여부']
else:
    feature_list = ['소속학과', '학년', '성별', '나이', '거주지', '학점', 'F학점과목수', '결석수', '장학건수',
                              '마일리지점수', '지도교수상담횟수', '전문가상담횟수', '대학생활적응분류', '중도탈락여부']

analData = rawData_e[feature_list]
analData

,소속학과,학년,성별,나이,거주지,학점,F학점과목수,결석수,장학건수,마일리지점수,지도교수상담횟수,전문가상담횟수,대학생활적응분류,중도탈락여부
0,0,1,0,23,1,3.06,0,0.000,0,27,3,0,2,0
1,2,0,1,23,3,4.45,0,0.103,3,1,0,1,2,0
2,3,1,0,23,4,3.40,0,0.400,0,6,2,0,2,1
3,2,1,0,23,5,3.38,0,1.083,0,25,2,1,2,0
4,4,1,0,23,5,2.50,0,0.143,0,13,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6092,7,1,1,20,66,2.92,0,1.105,0,19,4,1,0,0
6093,0,1,0,22,13,2.55,0,0.421,0,12,3,0,2,0
6094,0,1,0,22,62,2.50,0,1.053,2,12,3,0,1,0
6095,0,1,0,22,21,4.50,0,0.000,3,7,1,1,2,0


### (1-3-2) 분석대상데이터셋을 트레이닝셋과 테스트셋으로 분리
* 전처리된 데이터셋을 독립변수 데이터셋과 종속변수 데이터셋으로 분리
* 트레이닝셋과 테스트셋 비율을 7:3으로 하여 분석대상데이터셋 분리

In [ ]:
# 예측모형구축
# 소스 데이터프레임에서 분류(classification)을 위한 속성 집합
X = analData.loc[:, feature_list[:-1]]  # 독립변수 데이터셋
y = analData.loc[:, '중도탈락여부']  # 종속변수 데이터셋

# 트레이닝셋과 테스트셋 분리에 관련된 모듈 임포트
from sklearn.model_selection import train_test_split

# 자동으로 데이터셋을 트레이닝셋과 테스트셋으로 분리해주는 함수로
# 트레이닝셋과 데이터셋의 비율을 7:3으로 세팅함
# X_train : 트레이닝셋의 독립변수 데이터셋, y_train : 트레이닝셋의 종속변수 데이터셋
# X_test : 테스트셋의 독립변수 데이터셋, y_test : 테스트셋의 종속변수 데이터셋
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

### (1-3-3) 종속변수인 중도탈락여부 클래스의 분포 불균형 해소
* 중도탈락여부 변수값은 중도탈락(1)과 학업유지(0) 등 2가지 종류가 있는데, ‘학업유지’ 값을 갖는 경우가 ‘중도탈락’ 값을 갖는 경우보다 월등히 많아 분류성능지표(정확도 등)를 왜곡할 수 있음
* 이를 해결하기 위해서 적은 분포를 갖는 클래스의 투플을 인공적으로 만들어 클래스 분포의 균형을 이루는 오버샘플링(oversampling) 기법을 적용함. 대표적으로 SMOTE(Synthetic Minority Oversampling Technique) 기법이 있음
 * SMOTE(Synthetic Minority Oversampling Technique) : 데이터의 개수가 적은 클래스의 표본을 가져온 뒤 임의의 값을 추가하여 새로운 샘플을 만들어 데이터에 추가하는 오버샘플링 방식

In [ ]:
# SMOTE기법(Over-sampling) 적용을 위한 패키지(모듈) 임포트
from imblearn.over_sampling import SMOTE

# 트레이닝셋에 대한 SMOTE 적용
smote = SMOTE(random_state=0)
X_train_over,y_train_over = smote.fit_resample(X_train,y_train)

print('SMOTE 적용 전 트레이닝셋의 행열 갯수: ', X_train.shape, y_train.shape)
print('SMOTE 적용 후 트레이닝셋의 행열 갯수: ', X_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 트레이닝셋의 종속변수 값 분포: \n', pd.Series(y_train_over).value_counts())

# 테스트셋에 대한 SMOTE 적용
X_test_over,y_test_over = smote.fit_resample(X_test,y_test)
print('SMOTE 적용 전 테스트셋의 행열 갯수: ', X_test.shape, y_test.shape)
print('SMOTE 적용 후 테스트셋의 행열 갯수: ', X_test_over.shape, y_test_over.shape)
print('SMOTE 적용 후 테스트셋의 종속변수 값 분포: \n', pd.Series(y_test_over).value_counts())

SMOTE 적용 전 트레이닝셋의 행열 갯수:  (4267, 13) (4267,)
SMOTE 적용 후 트레이닝셋의 행열 갯수:  (7142, 13) (7142,)
SMOTE 적용 후 트레이닝셋의 종속변수 값 분포: 
 1    3571
0    3571
Name: 중도탈락여부, dtype: int64
SMOTE 적용 전 테스트셋의 행열 갯수:  (1830, 13) (1830,)
SMOTE 적용 후 테스트셋의 행열 갯수:  (3042, 13) (3042,)
SMOTE 적용 후 테스트셋의 종속변수 값 분포: 
 1    1521
0    1521
Name: 중도탈락여부, dtype: int64


* SMOTE 적용 후 트레이닝셋의 1(중도탈락)군과 0(학업유지)군의 건수 분포는 각각 3,571건으로 동일함
* SMOTE 적용 후 테스트셋의 1(중도탈락)군과 0(학업유지)군의 건수 분포는 각각 1,521건으로 동일함

### (1-3-4) 트레이닝셋을 대상으로 분류예측알고리듬 별로 학습을 수행하고 성능 측정/비교하여 가장 우수한 알고리듬 선택
* 비교 대상 분류예측알고리듬 : 의사결정트리(decision tree), 랜덤포레스트(random forest), XGBoost, LightGBM

In [ ]:
# 분류예측 성능측정을 위한 패키지(모듈) 임포트
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 트레이닝셋 학습 및 테스트셋 검증
def f_model_fit(cls, model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)   # 트레이닝셋 학습
    pred = model.predict(X_test)  # 테스트셋 검증
    f_metrics(cls, y_test, pred)  # 성능 측정

# 분류예측 성능측정
def f_metrics(cls, y_test, pred):
    accuracy = accuracy_score(y_test, pred)    # 정확도(accuracy) 측정
    precision = precision_score(y_test, pred)  # 정밀도(precision) 측정
    recall = recall_score(y_test, pred)        # 재현율(recall) 측정
    f1 = f1_score(y_test, pred)                # f1-score 측정
    auc = roc_auc_score(y_test, pred)          # auc 측정
    print(cls + '정확도 : {0:.3f}, 정밀도 : {1:.3f}, 재현율 : {2:.3f}, f1-score : {3:.3f}, AUC : {4:.3f}'.format(accuracy, precision, recall, f1, auc))

# 분석기법 : 의사결정트리(Decision_tree)
from sklearn.tree import DecisionTreeClassifier   # 의사결정트리 기법에 관련된 모듈 임포트

# DecisionTreeClassifier() : 의사결정트리 모형설정 함수
# 하이퍼패러미터(hyper parameter)
# criterion : 노드 분할을 위한 품질측정방법 (gini(default), log_loss, entropy 중 택1)
# max_depth : 트리의 최대 깊이 (default : None)
# random_state : 의사결정트리 생성 시 활용하는 데이터 패턴 지정. None(default)이면 트리를 만들 때마다 활용 데이터패턴이 달라져
# 결과가 달라지며, 특정 숫자를 준다면 활용 데이터패턴이 일정하여 동일한 결과를 냄
decision_tree = DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=0)
f_model_fit('Decision_tree(SMOTE적용 전) -> ', decision_tree, X_train, X_test, y_train, y_test)
f_model_fit('Decision_tree(SMOTE적용 후) -> ', decision_tree, X_train_over, X_test_over, y_train_over, y_test_over)
print('\n')

# 분석기법 : 랜덤포레스트(Random_forest Classification)
from sklearn.ensemble import RandomForestClassifier  # 랜덤포레스트 기법에 관련된 모듈

# RandomForestClassifier() : 랜덤포레스트 모형설정 함수
# 하이퍼패러미터(hyper parameter)
# n_estimators : 모델에서 사용할 트리 갯수(학습시 생성할 트리 갯수)
# random_state : 의사결정트리 생성 시 활용하는 데이터 패턴 지정. None(default)이면 트리를 만들 때마다 활용 데이터패턴이 달라져
# 결과가 달라지며, 특정 숫자를 준다면 활용 데이터패턴이 일정하여 동일한 결과를 냄
random_forest = RandomForestClassifier(n_estimators=20, random_state=0)
f_model_fit('Random_forest(SMOTE적용 전) -> ', random_forest, X_train, X_test, y_train, y_test)
f_model_fit('Random_forest(SMOTE적용 후) -> ', random_forest, X_train_over, X_test_over, y_train_over, y_test_over)
print('\n')

# 분석기법 : XGBoost
import xgboost as xgb # XGBoost 기법에 관련된 모듈

# XGBClassifier() : XGBoost 모형설정 함수
# 하이퍼패러미터(hyper parameter)
# max_depth : 트리의 최대 깊이(default:6)
# n_estimators : 모델에서 사용할 트리 갯수(학습시 생성할 트리 갯수)
# learning_rate : 학습 단계별로 이전의 결과 가중치를 얼만큼 사용할지 결정
# eval_metric : 오차 계산 방식 (rmse, mae, logloss, error, merror, mlogloss, auc)
gbm = xgb.XGBClassifier(max_depth=4, n_estimators=500, learning_rate=0.05, eval_metric='logloss', use_label_encoder=False)
f_model_fit('XGBoost(SMOTE적용 전) -> ', gbm, X_train, X_test, y_train, y_test)
f_model_fit('XGBoost(SMOTE적용 후) -> ', gbm, X_train_over, X_test_over, y_train_over, y_test_over)
print('\n')

# 분석기법 : LightGBM
from lightgbm import LGBMClassifier  # LightGBM 기법에 관련된 모듈

# LGBMClassifier() : LightGBM 모형설정 함수
# 하이퍼패러미터(hyper parameter)
# n_estimators : 모델에서 사용할 트리 갯수(학습시 생성할 트리 갯수)
# n_jobs : 학습을 위해 사용할 스레드 개수(-1은 모든 스레드를 사용함을 의미)
# boost_from_average : 불균형한 데이터 세트에서 예측 성능이 매우 저조할 경우 False로 설정 필요
lgb = LGBMClassifier(n_estimators=1000,num_leaves=64,n_jobs=-1,boost_from_average=False)
f_model_fit('LightGBM(SMOTE적용 전) -> ', lgb, X_train, X_test, y_train, y_test)
f_model_fit('LightGBM(SMOTE적용 후) -> ', lgb, X_train_over, X_test_over, y_train_over, y_test_over)

Decision_tree(SMOTE적용 전) -> 정확도 : 0.840, 정밀도 : 0.567, 재현율 : 0.233, f1-score : 0.330, AUC : 0.598
Decision_tree(SMOTE적용 후) -> 정확도 : 0.808, 정밀도 : 0.861, 재현율 : 0.734, f1-score : 0.792, AUC : 0.808


Random_forest(SMOTE적용 전) -> 정확도 : 0.842, 정밀도 : 0.553, 재현율 : 0.320, f1-score : 0.406, AUC : 0.634
Random_forest(SMOTE적용 후) -> 정확도 : 0.808, 정밀도 : 0.857, 재현율 : 0.738, f1-score : 0.793, AUC : 0.808


XGBoost(SMOTE적용 전) -> 정확도 : 0.838, 정밀도 : 0.531, 재현율 : 0.333, f1-score : 0.410, AUC : 0.637
XGBoost(SMOTE적용 후) -> 정확도 : 0.840, 정밀도 : 0.859, 재현율 : 0.813, f1-score : 0.836, AUC : 0.840


LightGBM(SMOTE적용 전) -> 정확도 : 0.837, 정밀도 : 0.522, 재현율 : 0.379, f1-score : 0.439, AUC : 0.654
LightGBM(SMOTE적용 후) -> 정확도 : 0.821, 정밀도 : 0.874, 재현율 : 0.749, f1-score : 0.807, AUC : 0.821


* 오버샘플링(SMOTE)을 적용한 결과가 그렇지 않은 결과에 비해 전반적으로 우수하고, 특히 재현율과 정밀도에 있어서 큰 효과를 보임
* 4개의 알고리듬 중에는 XGBoost가 가장 우수한 성능을 보임. 특히, 주요하게 봐야할 지표인 재현율(실제 중도탈락한 학생이 중도탈락으로 예측된 비율)에서 최고의 성능을 보였으며 AUC 값도 가장 우수함
* 그러므로, 성능측정 결과 XGBoost를 분류예측 알고리듬으로 채택하는 것이 바람직함

### (1-3-5) 독립변수 별 중도탈락예측 영향도 파악
* 중도탈락위험률 산정에 각 독립변수들이 어느 정도 영향을 미쳤는지에 대한 정량화된 수치로 중도탈락 예방을 위한 학생 관리 및 지원 프로그램에 대한 시사점(insight) 도출에 활용
* 성능측정 결과로 채택된 알고리듬 XGBoost를 기준으로 영향도가 큰 독립변수부터 출력

In [ ]:
# 속성(feature) 별 영향도를 저장하는 데이터프레임 생성
feature_imp_xgboost = pd.DataFrame({'영향도(xgboost)' : gbm.feature_importances_}, index = feature_list[:-1])
feature_imp_dtree = pd.DataFrame({'영향도(dtree)' : decision_tree.feature_importances_}, index = feature_list[:-1])
feature_imp_rforest = pd.DataFrame({'영향도(rforest)' : random_forest.feature_importances_}, index = feature_list[:-1])
feature_imp_lgbm = pd.DataFrame({'영향도(lgbm)' : lgb.feature_importances_}, index = feature_list[:-1])

feature_imp = pd.merge(feature_imp_xgboost, feature_imp_dtree, left_index=True, right_index=True, how='inner')
feature_imp = pd.merge(feature_imp, feature_imp_rforest, left_index=True, right_index=True, how='inner')
feature_imp = pd.merge(feature_imp, feature_imp_lgbm, left_index=True, right_index=True, how='inner')
feature_imp.sort_values('영향도(xgboost)', ascending=False)

,영향도(xgboost),영향도(dtree),영향도(rforest),영향도(lgbm)
성별,0.455063,0.483011,0.177083,1144
학년,0.196185,0.021947,0.068997,373
나이,0.084063,0.219438,0.074884,1992
전문가상담횟수,0.066556,0.090138,0.063395,1369
결석수,0.036607,0.086142,0.147787,9988
장학건수,0.022779,0.000000,0.043781,3051
마일리지점수,0.022698,0.037628,0.092771,10006
대학생활적응분류,0.022513,0.000000,0.025205,2120
학점,0.021684,0.058730,0.097046,11404
지도교수상담횟수,0.020746,0.000000,0.038324,2178


* XGBoost를 기준으로 가장 영향도가 큰 독립변수는 성별로 나타남 -> 중도탈락위험률이 상대적으로 높은 남학생들을 위한 지원프로그램 강화 모색 필요 (예, 군복무 기간 중 관리방안 등)
* 독립변수 별 중도탈락 영향도는 분류·예측 알고리듬에 따라 다르게 나타날 수 있기 때문에 절대적으로 해석하기 보다 참고정보로 활용하는 것이 바람직함. 다만, 여러 알고리듬에서 공통적으로 영향도가 높게 나타난다면, 충분히 유의미할 수 있기 때문에 중요 영향변수로 간주하고 학생지원 프로그램 설계에 반영할 필요 있음

# 2. 예측대상 데이터 수집 및 예측모형 적용
## 2-1) 현(現) 재학생 데이터 수집
* 데이터전처리 레이블인코딩 파트에서 NULL값을 가진 로우를 제거하고 범주형 특성을 레이블인코딩한 예측용 현 재학생 데이터셋 활용
* 현 재학생은 2023학년도 9월 1일 기준 재학생(2년제 1학년, 3년제 1,2학년) 기준이고 중도탈락여부는 익년 4월 1일 기준 예측이 목적임

## 2-2) 예측모형 적용
(1) 현(現) 재학생 데이터셋을 대상으로 예측용 데이터셋 확보<br>
(2) 예측용 데이터셋을 예측모형에 적용하여 재학생별 중도탈락위험률과 중도탈락위험순위 구하기<br>
(3) 현 재학생 중도탈락위험 예측결과와 실제 중도탈락여부(익년 4/1일 기준) 비교<br>
(4) 현 재학생 중도탈락위험 예측결과를 엑셀(excel)로 저장<br>

### (2-2-1) 현(現) 재학생 데이터셋을 대상으로 예측용 데이터셋 확보
* 현(現) 재학생의 예측대상 원천데이터셋이 확보되면 예측모형 개발과정에서 적용했던 데이터전처리 과정을 거쳐 예측모형 적용대상 데이터셋을 확보함

In [ ]:
# 전처리된 현 재학생 데이터셋을 저장하는 데이터프레임 확인
rawData2_e

,학생ID,소속학과,학년,성별,나이,거주지,학점,F학점과목수,결석수,장학건수,마일리지점수,지도교수상담횟수,전문가상담횟수,대학생활적응분류,중도탈락여부
0,25.0,17,1,0,25,16,3.28,0,0.350,1,3,0,1,1,0
1,7315.0,18,0,0,21,1,3.95,0,0.000,2,3,1,0,1,0
2,9249.0,6,1,0,21,11,2.76,0,0.526,2,13,2,1,2,1
3,6324.0,26,0,1,21,6,0.61,6,2.350,1,4,1,1,0,1
4,7570.0,6,1,1,22,3,0.24,5,5.211,2,8,2,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2983,2312.0,4,1,0,23,23,3.57,0,0.000,0,23,2,0,0,0
2984,6658.0,4,1,0,23,8,3.46,0,0.000,0,3,0,1,0,0
2985,4042.0,0,1,0,23,44,4.17,0,0.000,3,6,2,0,0,0
2986,6968.0,14,1,0,23,4,2.95,0,0.650,2,3,1,0,0,0


### (2-2-2) 예측용 데이터셋을 예측모형에 적용하여 재학생별 중도탈락위험률과 중도탈락위험순위 구하기
* 재학생 개인별 중도탈락예측 결과로 다음 항목을 도출할 수 있음
 * 재학생별 중도탈락위험률
   * 재학생별로 중도탈락 위험정도를 나타내는 백분위값
   * 예측모형의 함수를 활용하여 구할 수 있음
 * 재학생별 중도탈락위험순위
   * 학과/반별 중도탈락 위험순위 -> 순위가 높을수록 위험정도가 높으며, 반별 지도교수가 집중 지도 및 관리 재학생을 선별하는데 활용
   * 각 재학생들의 학생ID로 학번을 파악하고, 지도반별 재학생들의 중도탈락위험률을 내림차순 정렬하여 구할 수 있음

In [ ]:
# 분석대상 특성리스트 (독립변수만)
feature_ind_list = feature_list[:-1]

# 분석대상 특성으로 구성된 예측용 데이터셋
expData = rawData2_e[feature_ind_list]

# 예측용 데이터셋
expData

,소속학과,학년,성별,나이,거주지,학점,F학점과목수,결석수,장학건수,마일리지점수,지도교수상담횟수,전문가상담횟수,대학생활적응분류
0,17,1,0,25,16,3.28,0,0.350,1,3,0,1,1
1,18,0,0,21,1,3.95,0,0.000,2,3,1,0,1
2,6,1,0,21,11,2.76,0,0.526,2,13,2,1,2
3,26,0,1,21,6,0.61,6,2.350,1,4,1,1,0
4,6,1,1,22,3,0.24,5,5.211,2,8,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2983,4,1,0,23,23,3.57,0,0.000,0,23,2,0,0
2984,4,1,0,23,8,3.46,0,0.000,0,3,0,1,0
2985,0,1,0,23,44,4.17,0,0.000,3,6,2,0,0
2986,14,1,0,23,4,2.95,0,0.650,2,3,1,0,0


In [ ]:
# predict() : 모형변수의 함수로서 신입생 각각에 대한 중도탈락여부 예측
y_pred_dtree = decision_tree.predict(expData)    # 의사결정트리
y_pred_rforest = random_forest.predict(expData)  # 랜덤포레스트
y_pred_xgboost = gbm.predict(expData)            # XGBoost
y_pred_lgbm = lgb.predict(expData)               # LightGBM

# predict_proba() : 모형변수의 함수로서 신입생 각각에 대한 중도탈락위험률 예측
y_pred_prob_dtree = decision_tree.predict_proba(expData)     # 의사결정트리
y_pred_prob_rforest = random_forest.predict_proba(expData)   # 랜덤포레스트
y_pred_prob_xgboost = gbm.predict_proba(expData)             # GBoost
y_pred_prob_lgbm = lgb.predict_proba(expData)                # LightGBM

In [ ]:
expData.loc[:, '중도탈락여부예측(dtree)'] = y_pred_dtree
expData.loc[:, '중도탈락위험률(dtree)'] = y_pred_prob_dtree[:,1]

expData.loc[:, '중도탈락여부예측(rforest)'] = y_pred_rforest
expData.loc[:, '중도탈락위험률(rforest)'] = y_pred_prob_rforest[:,1]

expData.loc[:, '중도탈락여부예측(xgboost)'] = y_pred_xgboost
expData.loc[:, '중도탈락위험률(xgboost)'] = y_pred_prob_xgboost[:,1]

expData.loc[:, '중도탈락여부예측(lgbm)'] = y_pred_lgbm
expData.loc[:, '중도탈락위험률(lgbm)'] = y_pred_prob_lgbm[:,1]

expData

C:\Users\a\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\a\anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


,소속학과,학년,성별,나이,거주지,학점,F학점과목수,결석수,장학건수,마일리지점수,...,전문가상담횟수,대학생활적응분류,중도탈락여부예측(dtree),중도탈락위험률(dtree),중도탈락여부예측(rforest),중도탈락위험률(rforest),중도탈락여부예측(xgboost),중도탈락위험률(xgboost),중도탈락여부예측(lgbm),중도탈락위험률(lgbm)
0,17,1,0,25,16,3.28,0,0.350,1,3,...,1,1,0,0.433476,1,0.75,1,0.708098,1,9.710842e-01
1,18,0,0,21,1,3.95,0,0.000,2,3,...,0,1,0,0.433476,0,0.20,0,0.106543,0,9.604929e-07
2,6,1,0,21,11,2.76,0,0.526,2,13,...,1,2,0,0.101818,0,0.05,0,0.096150,0,7.947195e-05
3,26,0,1,21,6,0.61,6,2.350,1,4,...,1,0,0,0.327660,0,0.45,0,0.352065,0,6.486094e-03
4,6,1,1,22,3,0.24,5,5.211,2,8,...,0,2,0,0.162896,1,0.80,1,0.661392,1,9.977778e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2983,4,1,0,23,23,3.57,0,0.000,0,23,...,0,0,0,0.101818,0,0.05,0,0.010677,0,1.111387e-10
2984,4,1,0,23,8,3.46,0,0.000,0,3,...,1,0,0,0.433476,0,0.20,0,0.401553,0,4.094874e-04
2985,0,1,0,23,44,4.17,0,0.000,3,6,...,0,0,0,0.433476,0,0.05,0,0.005156,0,1.004478e-13
2986,14,1,0,23,4,2.95,0,0.650,2,3,...,0,0,0,0.433476,0,0.30,0,0.271527,0,2.287209e-03


### (2-2-3) 현 재학생 중도탈락위험 예측결과와 실제 중도탈락여부(익년 4/1일 기준) 비교
* 현 재학생 데이터셋에 당해년도 9/1일 기준 중도탈락여부가 존재하므로 예측결과와 비교

In [ ]:
# 예측용 데이터셋에 학생ID와 실제 중도탈락여부(9/1일기준) 컬럼 추가
expData['학생ID'] = rawData2_e['학생ID']
expData['중도탈락여부'] = rawData2_e['중도탈락여부']

In [ ]:
# 예측결과와 실제결과가 일치하는 건수를 알고리듬별로 저장하는 변수 초기화
cnt_df = 0
cnt_rf = 0
cnt_xg = 0
cnt_lg = 0

# 일치여부 확인
for i in range(0, len(expData)) :
    if expData['중도탈락여부'].values[i] == expData['중도탈락여부예측(dtree)'].values[i] :
        cnt_df += 1

    if expData['중도탈락여부'].values[i] == expData['중도탈락여부예측(rforest)'].values[i] :
        cnt_rf += 1

    if expData['중도탈락여부'].values[i] == expData['중도탈락여부예측(xgboost)'].values[i] :
        cnt_xg += 1

    if expData['중도탈락여부'].values[i] == expData['중도탈락여부예측(lgbm)'].values[i] :
        cnt_lg += 1

# 결과 출력
print("Decision tree : 적중개수 %d, 적중률 %.2f" % (cnt_df, cnt_df / len(expData) * 100))
print("Random Forest : 적중개수 %d, 적중률 %.2f" % (cnt_rf, cnt_rf / len(expData) * 100))
print("XGBoost : 적중개수 %d, 적중률 %.2f" % (cnt_xg, cnt_xg / len(expData) * 100))
print("Light GBM : 적중개수 %d, 적중률 %.2f" % (cnt_lg, cnt_lg / len(expData) * 100))

Decision tree : 적중개수 2292, 적중률 76.71
Random Forest : 적중개수 2233, 적중률 74.73
XGBoost : 적중개수 2170, 적중률 72.62
Light GBM : 적중개수 2191, 적중률 73.33


In [ ]:
expData

,소속학과,학년,성별,나이,거주지,학점,F학점과목수,결석수,장학건수,마일리지점수,...,중도탈락여부예측(dtree),중도탈락위험률(dtree),중도탈락여부예측(rforest),중도탈락위험률(rforest),중도탈락여부예측(xgboost),중도탈락위험률(xgboost),중도탈락여부예측(lgbm),중도탈락위험률(lgbm),학생ID,중도탈락여부
0,17,1,0,25,16,3.28,0,0.350,1,3,...,0,0.433476,1,0.75,1,0.708098,1,9.710842e-01,25.0,0
1,18,0,0,21,1,3.95,0,0.000,2,3,...,0,0.433476,0,0.20,0,0.106543,0,9.604929e-07,7315.0,0
2,6,1,0,21,11,2.76,0,0.526,2,13,...,0,0.101818,0,0.05,0,0.096150,0,7.947195e-05,9249.0,1
3,26,0,1,21,6,0.61,6,2.350,1,4,...,0,0.327660,0,0.45,0,0.352065,0,6.486094e-03,6324.0,1
4,6,1,1,22,3,0.24,5,5.211,2,8,...,0,0.162896,1,0.80,1,0.661392,1,9.977778e-01,7570.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2983,4,1,0,23,23,3.57,0,0.000,0,23,...,0,0.101818,0,0.05,0,0.010677,0,1.111387e-10,2312.0,0
2984,4,1,0,23,8,3.46,0,0.000,0,3,...,0,0.433476,0,0.20,0,0.401553,0,4.094874e-04,6658.0,0
2985,0,1,0,23,44,4.17,0,0.000,3,6,...,0,0.433476,0,0.05,0,0.005156,0,1.004478e-13,4042.0,0
2986,14,1,0,23,4,2.95,0,0.650,2,3,...,0,0.433476,0,0.30,0,0.271527,0,2.287209e-03,6968.0,0


* 분석차원(독립변수)를 줄였을 때는 Random Forest가 그렇지 않았을 때는 XGBoost의 적중률이 가장 높았으며, 전체적으로는 차원을 줄이지 않은 XGBoost의 적중률이 가장 높았음

In [ ]:
result_df = pd.merge(rawData2_org, expData[['학생ID','중도탈락여부예측(dtree)','중도탈락위험률(dtree)'
                                            ,'중도탈락여부예측(rforest)','중도탈락위험률(rforest)'
                                            ,'중도탈락여부예측(xgboost)','중도탈락위험률(xgboost)'
                                            ,'중도탈락여부예측(lgbm)','중도탈락위험률(lgbm)']], how='left', on='학생ID')
result_df

,학생ID,소속학과,학년,성별,나이,거주지,학점,F학점과목수,결석수,장학건수,...,중도탈락여부,구분,중도탈락여부예측(dtree),중도탈락위험률(dtree),중도탈락여부예측(rforest),중도탈락위험률(rforest),중도탈락여부예측(xgboost),중도탈락위험률(xgboost),중도탈락여부예측(lgbm),중도탈락위험률(lgbm)
0,25.0,학과18,2,남,25,거주016,3.28,0,0.350,1,...,0.0,예측용,0.0,0.433476,1.0,0.75,1.0,0.708098,1.0,9.710842e-01
1,7315.0,학과19,1,남,21,거주001,3.95,0,0.000,2,...,0.0,예측용,0.0,0.433476,0.0,0.20,0.0,0.106543,0.0,9.604929e-07
2,4918.0,학과29,2,여,21,거주059,3.24,0,1.571,0,...,0.0,예측용,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9249.0,학과07,2,남,21,거주011,2.76,0,0.526,2,...,1.0,예측용,0.0,0.101818,0.0,0.05,0.0,0.096150,0.0,7.947195e-05
4,3418.0,학과10,1,남,21,거주008,1.20,4,3.476,1,...,0.0,예측용,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3186,2312.0,학과05,2,남,23,거주023,3.57,0,0.000,0,...,0.0,예측용,0.0,0.101818,0.0,0.05,0.0,0.010677,0.0,1.111387e-10
3187,6658.0,학과05,2,남,23,거주008,3.46,0,0.000,0,...,0.0,예측용,0.0,0.433476,0.0,0.20,0.0,0.401553,0.0,4.094874e-04
3188,4042.0,학과01,2,남,23,거주044,4.17,0,0.000,3,...,0.0,예측용,0.0,0.433476,0.0,0.05,0.0,0.005156,0.0,1.004478e-13
3189,6968.0,학과15,2,남,23,거주004,2.95,0,0.650,2,...,0.0,예측용,0.0,0.433476,0.0,0.30,0.0,0.271527,0.0,2.287209e-03


### (2-2-4) 현 입학초신입생별 중도탈락위험 예측결과를 엑셀(excel)로 저장

In [ ]:
# XlsxWriter 엔진으로 Pandas writer 객체 만들기
if feature_set_shrink == 1:  # 차원축소한 경우
    writer = pd.ExcelWriter('2023재학생별_중도탈락위험예측(차원축소)결과.xlsx')
else: # 차원축소하지 않은 경우
    writer = pd.ExcelWriter('2023재학생별_중도탈락위험예측결과.xlsx')

# 데이터프레임 내용을 엑셀에 저장
result_df.to_excel(writer, sheet_name='예측결과')

# Pandas writer 객체 닫기
writer.close()